In [30]:
from selenium import webdriver
from time import sleep
import threading
import re

In [50]:
def get_item(page, driver): 
    
    # regex of item number
    regex = re.compile('[0-9]+\?page')
    
    driver.get("http://getitbeauty.m.tving.com/Product?page={}".format(page))
    sleep(0.5)

    lis = driver.find_elements_by_css_selector('.subType5 li dl') 

    
    idx = 0
    rows = []
    while(True):
        try: 
            li = lis[idx]
            
            img = li.find_element_by_css_selector('.thumb a img').get_attribute('src')
            name = li.find_element_by_css_selector('dt a').text

            brand = li.find_element_by_css_selector('dt a strong').text

            # split category
            cates = li.find_elements_by_css_selector('.gray a')
            cate_res = []
            for cate in cates:
                cate_res.append(cate.text)


            link = li.find_element_by_css_selector('.thumb a').get_attribute('href')

            # split item number
            matchobj = regex.search(link)
            item_num = matchobj.group()
            item_num = item_num.split('?')[0] 
             
            rows.append([item_num, img, name, brand, cate_res, link])
            
            idx = idx + 1
            if idx == len(lis):
                break
                
        except Exception as e:
            del rows[:]
            rows = [] 
            sleep(1) 
            
            lis = driver.find_elements_by_css_selector('.subType5 li dl') 
            idx = 0 
         
         
    return rows


def get_page(start, end):
    
    driver =  webdriver.PhantomJS()
    for page in range(start, end):
        page_items = get_item(page, driver)
        all_items.extend(page_items)
        
    driver.close() 

In [51]:
!killall phantomjs

No matching processes belonging to you were found


431페이지마다 12개 아이템이 있으며 마지막 432페이즈는 8개만 있다.

In [78]:
(431 * 12) + 8

5180

스레드 6개로 분할하여 72페이지씩 크롤링 작업 수행

In [49]:
432 / 6

72.0

In [52]:
all_items = []
for cnt in range(1, 7): 
    start = 72 * (cnt-1)+1
    end = 72 * cnt+1 
    
    th = threading.Thread(target=get_page, args=(start, end))
    th.start()

In [77]:
len(all_items)

5180

In [83]:
item_df = pd.DataFrame(all_items, columns=['id_item','link','name','brand','category','img'])

In [89]:
item_df['cate1']= item_df['category'].apply(lambda x: x[0])
item_df['cate2']= item_df['category'].apply(lambda x: x[1])

In [92]:
item_df = item_df.drop(columns=['category'])

In [94]:
item_df.to_csv('data/item.csv', index=False)